# Preliminary Data Analysis

Some preliminary data analysis on the data acquired during regen testing on July 6th, between 22:02 and 22:04 (PST).
This notebook takes advantage of the custom `TimeSeries` class, which is a data-analysis centric wrapper around `np.ndarray`.

In [20]:
from data_analysis.data_tools.time_series import TimeSeries
from data_analysis.data_tools.influx_client import InfluxClient

from simulation.common import BrightSide

import matplotlib.pyplot as plt
import numpy as np

# 1st Run
start = "2024-07-07T05:02:19Z"
stop = "2024-07-07T05:04:04Z"

client = InfluxClient()

Query the data, then align their x–axes such that math is well-defined.

In [33]:
pack_voltage: TimeSeries = client.query_time_series(start, stop, "TotalPackVoltage", units="V")
pack_current: TimeSeries = client.query_time_series(start, stop, "PackCurrent", units="A")
vehicle_velocity: TimeSeries = client.query_time_series(start, stop, "VehicleVelocity", units="m/s")
pack_current, pack_voltage, vehicle_velocity = TimeSeries.align(pack_current, pack_voltage, vehicle_velocity)

# Subtract LVS current from pack current
pack_current -= BrightSide.lvs_current

Plot the data to make sure nothing bad or spooky happened (and matches what's seen on InfluxDB Data Explorer).

In [ ]:
pack_voltage.plot(False)
pack_current.plot(False)
vehicle_velocity.plot(False)
plt.show()

Now, let's find the time that current is negative (regen is occurring).

In [ ]:
# Find indices where regen occurs
current_negative_mask = np.where(pack_current < -0.5)[0]

# Group consecutive indices to identify regen periods
regen_periods = []
current_period = [current_negative_mask[0]] # Start the first regen period with the first negative current index

for i in range(1, len(current_negative_mask)):
    # If indices are consecutive, extend the current_period
    if current_negative_mask[i] == current_negative_mask[i - 1] + 1:
        current_period.append(current_negative_mask[i]) 
    # If indices are not consecutive, add the list of current_period to regen_periods
    else:
        regen_periods.append(current_period) 
        current_period = [current_negative_mask[i]] # Start a new current_period with index i

regen_periods.append(current_period)  # Add the last current_period to the list of regen periods 

for period in regen_periods:
    # We find the indices, as slicing pack_current with current_negative_mask isn't a well-defined operation
    first_regen_index = period[0]
    last_regen_index = period[-1]

    regen_current = pack_current[first_regen_index:last_regen_index]
    regen_voltage = pack_voltage[first_regen_index:last_regen_index]
    regen_current.plot(show=False)
    regen_voltage.plot(show=False)
    plt.show()

We can also inspect the change in velocity to get a sense of the change in kinetic energy, estimating a vehicle mass of 300kg.

In [ ]:
period_index = 1 # Keep track of index while printing velocities, energies and efficiencies

for period in regen_periods:
    initial_velocity = vehicle_velocity[first_regen_index]                              # Units: m/s
    final_velocity = vehicle_velocity[last_regen_index]                                 # Units: m/s
    kinetic_energy  = lambda vi, vf: 0.5 * BrightSide.vehicle_mass * (vf**2 - vi**2)    # Units: J
    delta_kinetic_energy = -kinetic_energy(final_velocity, initial_velocity)            # Units: J

    print(f"Initial Velocity for Period {period_index}: {initial_velocity:.4f} {vehicle_velocity.units}")
    print(f"Final Velocity for Period {period_index}: {final_velocity:.4f} {vehicle_velocity.units}")
    print(f"Change in Kinetic Energy for Period {period_index}: {delta_kinetic_energy:.4f} J")

    period_index += 1

Now, let's calculate the battery power during regen.
Note: the elementwise-array multiplication returns an `ndarray`, so I use `regen_current.promote()` on the result to get back a `TimeSeries`.

In [ ]:
for period in regen_periods:  
    regen_power = regen_current.promote(regen_current * regen_voltage)
    regen_power.units = "W"  # Fix units after math
    regen_power.meta["field"] = "Pack Power"
    regen_power.plot()

Now, let's integrate with respect to time to get regen energy, and then get efficiency.

In [ ]:
period_index = 1 # Keep track of index while printing velocities, energies and efficiencies

for period in regen_periods:
    regen_energy = np.trapz(regen_power, regen_power.x_axis)  # Units: J
    efficiency: float = regen_energy / delta_kinetic_energy

    print(f"Regen Energy for Period {period_index}: {regen_energy:.3f} J")
    print(f"Regen Efficiency for Period {period_index}: {efficiency * 100:.2f}%")

    period_index += 1